In [14]:
import fitz

In [54]:
# # PDF to Image, use OCR to detect bounding boxes of text
# from pdf2image import convert_from_path
# import easyocr
# import numpy as np
# pdf_path = 'Structured_note_termsheet/cct-linked-structured-note.pdf'
# images = convert_from_path(pdf_path)
# reader = easyocr.Reader(['en'])
# bounds = reader.readtext(np.array(images[0]), min_size=10, slope_ths=0.5, ycenter_ths=0.7, height_ths=1.2, width_ths=0.85, decoder='beamsearch', beamWidth=10)

In [56]:
# # export the image to a file
# images[0].save('output.jpg', 'JPEG')

In [36]:
# # Get the coordinates bounding box of the text in the PDF
# def search_text_and_get_coordinates(pdf_path, search_text):
#     document = fitz.open(pdf_path)
    
#     start_text_coordinates = (float('inf'), float('inf'))
#     end_text_coordinates = (float('-inf'), float('-inf'))
    
#     for page_number in range(len(document)):
#         page = document.load_page(page_number)
        
#         text_instances = page.search_for(search_text)
        
#         for inst in text_instances:
#             x0, y0, x1, y1 = inst
#             if start_text_coordinates is None:
#                 start_text_coordinates = (x0, y0)
#             else:
#               # choose the smallest x0 and y0
#               start_text_coordinates = (min(start_text_coordinates[0], x0), min(start_text_coordinates[1], y0))
        
#             if end_text_coordinates is None:
#                 end_text_coordinates = (x1, y1)
#             else:
#               # choose the largest x1 and y1
#               end_text_coordinates = (max(end_text_coordinates[0], x1), max(end_text_coordinates[1], y1))
              
#     return start_text_coordinates, end_text_coordinates

# pdf_path = 'Structured_note_termsheet/cct-linked-structured-note.pdf'

# search_text = 'Status of the Notes: The Notes are direct, unsecured and unsubordinated obligations of the Issuer and rank pari passu without any preference among themselves and (save for certain obligations required to be preferred by law) equally with all other unsecured obligations (other than subordinated obligations, if any) of the Issuer, from time to time outstanding.'

# search_text_and_get_coordinates(pdf_path, search_text)

((86.30400085449219, 280.5127868652344), (526.9910888671875, 340.22802734375))

In [42]:
# data_json = {
#   "training_data": [
#     {
#       "text": "Status of the Notes: The Notes are direct, unsecured and unsubordinated obligations of the Issuer and rank pari passu without any preference among themselves and (save for certain obligations required to be preferred by law) equally with all other unsecured obligations (other than subordinated obligations, if any) of the Issuer, from time to time outstanding.",
#       "start_text_coordinates": {
#         "x": 86.30400085449219,
#         "y": 280.5127868652344
#       },
#       "end_text_coordinates": {
#         "x": 526.9910888671875,
#         "y": 340.22802734375
#       },
#       "expected_output": "<StatusOfNotes><Description>The Notes are direct, unsecured and unsubordinated obligations of the Issuer and rank pari passu without any preference among themselves and (save for certain obligations required to be preferred by law) equally with all other unsecured obligations (other than subordinated obligations, if any) of the Issuer, from time to time outstanding.</Description></StatusOfNotes>"
#     }
#   ]
# }

# # tabulur data extraction 
# # key value pairs
# # page breaks

In [61]:
# PDF to Images
import fitz
import os

dpi = 300
zoom = dpi/72
magnify = fitz.Matrix(zoom, zoom)

folder_name = 'standard_settlement_instructions'
file_name = 'standard-settlement-instructions.pdf'
path = f"{folder_name}/{file_name}"
count = 0

doc = fitz.open(path)

for page in doc:
    count+=1
    pix = page.get_pixmap(matrix=magnify)
    pix.save(f"Images/{file_name[:-3]}_page_{count}.png")

In [80]:
# Using the Tesseract OCR engine to extract text from the images
import os
import json
import pytesseract
from PIL import Image
from pathlib import Path
from uuid import uuid4

# Set the Tesseract OCR executable path
pytesseract.pytesseract.tesseract_cmd = '/opt/homebrew/bin/tesseract'  # Update this path if necessary

# Tesseract output levels for the level of detail for the bounding boxes
LEVELS = {
    'page_num': 1,
    'block_num': 2,
    'par_num': 3,
    'line_num': 4,
    'word_num': 5
}

def create_image_url(filepath):
    """
    Label Studio requires image URLs, so this defines the mapping from filesystem to URLs
    if you use ./serve_local_files.sh <my-images-dir>, the image URLs are localhost:8081/filename.png
    Otherwise you can build links like /data/upload/filename.png to refer to the files
    """
    filename = os.path.basename(filepath)
    return f'http://localhost:8080/{filename}'

def convert_to_ls(image, tesseract_output, per_level='block_num'):
    """
    :param image: PIL image object
    :param tesseract_output: the output from tesseract
    :param per_level: control the granularity of bboxes from tesseract
    :return: tasks.json ready to be imported into Label Studio with "Optical Character Recognition" template
    """
    image_width, image_height = image.size
    per_level_idx = LEVELS[per_level]
    results = []
    all_scores = []
    for i, level_idx in enumerate(tesseract_output['level']):
        if level_idx == per_level_idx:
            bbox = {
                'x': 100 * tesseract_output['left'][i] / image_width,
                'y': 100 * tesseract_output['top'][i] / image_height,
                'width': 100 * tesseract_output['width'][i] / image_width,
                'height': 100 * tesseract_output['height'][i] / image_height,
                'rotation': 0
            }

            words, confidences = [], []
            for j, curr_id in enumerate(tesseract_output[per_level]):
                if curr_id != tesseract_output[per_level][i]:
                    continue
                word = tesseract_output['text'][j]
                confidence = tesseract_output['conf'][j]
                words.append(word)
                if confidence != '-1':
                    confidences.append(float(confidence / 100.))

            text = ' '.join(words).strip()
            if not text:
                continue
            region_id = str(uuid4())[:10]
            score = sum(confidences) / len(confidences) if confidences else 0
            bbox_result = {
                'id': region_id, 'from_name': 'bbox', 'to_name': 'image', 'type': 'rectangle',
                'value': bbox}
            transcription_result = {
                'id': region_id, 'from_name': 'transcription', 'to_name': 'image', 'type': 'textarea',
                'value': dict(text=[text], **bbox), 'score': score}
            results.extend([bbox_result, transcription_result])
            all_scores.append(score)

    return {
        'data': {
            'ocr': create_image_url(image.filename)
        },
        'predictions': [{
            'result': results,
            'score': sum(all_scores) / len(all_scores) if all_scores else 0
        }]
    }

tasks = []
# Collect the receipt images from the image directory
for f in Path('Images').glob('*.png'):
    with Image.open(f.absolute()) as image:
        tesseract_output = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)
        task = convert_to_ls(image, tesseract_output, per_level='block_num')
        tasks.append(task)

# Create a file to import into Label Studio
with open(f'JSON/{file_name[:-3]}_noPaddle.json', mode='w') as f:
    json.dump(tasks, f, indent=2)

In [86]:
import os
from paddleocr import PaddleOCR
from PIL import Image, ImageDraw, ImageFont
import json
from uuid import uuid4
import numpy as np

# loading the engine
# OCR enginer
ocr = PaddleOCR(use_angle_cls=False, 
                lang='en',
                  rec=False,
                ) # need to run only once to download and load model into memory 

images_folder_path  = "Images" 

def create_image_url(filename):
    """
    Label Studio requires image URLs, so this defines the mapping from filesystem to URLs
    if you use ./serve_local_files.sh <my-images-dir>, the image URLs are localhost:8081/filename.png
    Otherwise you can build links like /data/upload/filename.png to refer to the files
    """
    return f'http://localhost:8080/{filename}'

def convert_bounding_box(bounding_box):
    """Converts a bounding box of [x1, y1, x2, y2] into [x, y, height, width].

    Args:
    bounding_box: A list of four numbers, representing the x1, y1, x2, and y2
        coordinates of the bounding box.

    Returns:
    A list of four numbers, representing the x, y, height, and width of the
        bounding box.
    """

    x1, y1, x2, y2 = bounding_box
    x = min(x1, x2)
    y = min(y1, y2)
    width = x2 - x1
    height = y2 - y1

    return [x, y, width, height]

def extracted_tables_to_label_studio_json_file_with_paddleOCR(images_folder_path):
    label_studio_task_list = []
    for images in os.listdir(images_folder_path):
        if images.endswith('.png'):
            output_json = {}
            annotation_result = []

            output_json['data'] =  {"ocr":create_image_url(images)}
                    
            img = Image.open(f'Images/{images}')

            img = np.asarray(img)
            image_height, image_width = img.shape[:2]

            result = ocr.ocr(img,cls=False)

            #print(result)

            for output in result:

                for item in output: 
                    co_ord = item[0]
                    text = item[1][0]

                    four_co_ord = [co_ord[0][0],co_ord[1][1],co_ord[2][0]-co_ord[0][0],co_ord[2][1]-co_ord[1][1]]

                    #print(four_co_ord)
                    #print(text)

                    bbox = {
                    'x': 100 * four_co_ord[0] / image_width,
                    'y': 100 * four_co_ord[1] / image_height,
                    'width': 100 * four_co_ord[2] / image_width,
                    'height': 100 * four_co_ord[3] / image_height,
                    'rotation': 0
                            }
                    

                    if not text:
                        continue
                    region_id = str(uuid4())[:10]
                    score = 0.5
                    bbox_result = {
                        'id': region_id, 'from_name': 'bbox', 'to_name': 'image', 'type': 'rectangle',
                        'value': bbox}
                    transcription_result = {
                        'id': region_id, 'from_name': 'transcription', 'to_name': 'image', 'type': 'textarea',
                        'value': dict(text=[text], **bbox), 'score': score}
                    annotation_result.extend([bbox_result, transcription_result])
                    #print('annotation_result :\n',annotation_result)
            output_json['predictions'] = [ {"result": annotation_result,  "score":0.97}]

            label_studio_task_list.append(output_json)
        
    # saving label_stdui_task_list as json file to import in label_studio
    with open(f'JSON/{file_name[:-3]}_Paddle.json', 'w') as f:
        json.dump(label_studio_task_list, f, indent=4)

extracted_tables_to_label_studio_json_file_with_paddleOCR(images_folder_path)

download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /Users/vincentzhao/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:24<00:00, 161kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /Users/vincentzhao/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:25<00:00, 399kiB/s] 

[2024/07/15 07:59:11] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/vincentzhao/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/vincentzhao/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6,

[2024/07/15 07:59:11] ppocr DEBUG: dt_boxes num : 76, elapsed : 0.27209019660949707
[2024/07/15 07:59:20] ppocr DEBUG: rec_res num  : 76, elapsed : 8.070325136184692


In [2]:
from unstructured.partition.pdf import partition_pdf 
from unstructured.staging.base import elements_to_json

/Users/vincentzhao/Desktop/Project_Master/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
include_page_breaks = True

## strategy
# The strategy to use for partitioning the PDF. Valid strategies are "hi_res", "ocr_only", and "fast".
# When using the "hi_res" strategy, the function uses a layout detection model to identify document elements.
# hi_res" is used for analyzing PDFs and extracting table structure (default is "auto")
strategy = "hi_res"

if strategy == "hi_res": infer_table_structure = True
else: infer_table_structure = False

if infer_table_structure == True: extract_element_types=['Table']
else: extract_element_types=None

if strategy != "ocr_only": max_characters = None

languages = ["eng"] # example if more than one "eng+por" (default is "eng")

## model_name
# @requires_dependencies("unstructured_inference")
# yolox: best model for table extraction. Other options are yolox_quantized, detectron2_onnx and chipper depending on file layout
# source: https://unstructured-io.github.io/unstructured/best_practices/models.html
hi_res_model_name = "yolox"

In [15]:
# requirement.txt:
# pillow
# unstructured[all-docs]==0.12.5
# pdfminer.six==20201018
# pillow-heif
# unstructured_inference
# opencv-python-headless
# opencv-python
# unstructured_pytesseract

In [4]:
filename = "standard_settlement_instructions/wfbna-usa-ssi-listing.pdf"

elements = partition_pdf(
        filename=filename,
        include_page_breaks=include_page_breaks,
        strategy=strategy,
        infer_table_structure=infer_table_structure,
        extract_element_types=extract_element_types,
        max_characters=max_characters,
        languages=languages,
        hi_res_model_name=hi_res_model_name,
)

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
from unstructured.staging.base import elements_to_json
elements_to_json(elements, filename=f"{filename}.json")

In [7]:
import json
import pathlib
from pathlib import Path

def process_json_file(input_filename):
    # Read the JSON file
    with open(input_filename, 'r') as file:
        data = json.load(file)

    # Iterate over the JSON data and extract required table elements
    extracted_elements = []
    text_prev = ""
    for i,entry in enumerate(data):
        if entry["type"] == "Title":
            text = "" + entry["text"] + ""
        elif entry["type"] == "Table":
            text = entry["metadata"]["text_as_html"]
        else:
            text = "" + entry["text"] + ""

        if text != text_prev: extracted_elements.append(text)
        text_prev = text

    # Write the extracted elements to the output file
    html_start = """
    
    
    
    Document Information
    
    
    
    """

    html_end = """
    
    
    """

    output_file_html = Path(input_filename).name.replace(".json", "") + ".html"
    with open(output_file_html, 'w') as output_file:
        output_file.write(html_start + "\n")
        for element in extracted_elements:
            output_file.write(element + "\n")
        output_file.write(html_end + "\n")

    return str(output_file_html)

output_file_html = process_json_file(f"{filename}.json") # It can take a while for the .html file to show up in Colab

In [ ]:
# Use llama to convert the HTML file to a xml file
